# T4 Combine Detection Results

- import the t1 inferenced point clouds
- import the t3 doors 
- export graphs

## LIBRARIES

In [26]:

#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

#import utils
import context 
import utils as utl
import utils.t1_utils as t1
import json

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

## INPUTS

In [5]:
#paths
path=Path(os.getcwd()).parents[2]
print(path)
input_folder_t1=path/'data'/'t1'/'input' # this should become t1_results
input_folder_t1_temp_graph=path/'data'/'t0'/'results' # this should become t1_results
input_folder_t2=path/'data'/'t2'
input_folder_t3=path/'data'/'t3'
class_file=path/'data'/'_classes.json'

output_folder=path/'data'/'t4'
os.makedirs(output_folder, exist_ok=True)

#parameters

c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge


### Import classes

In [40]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'windows', 'id': 5, 'temp_id': 6, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


### Import PCD

In [61]:
files=utl.get_list_of_files(input_folder_t1, '.laz')
files.extend(utl.get_list_of_files(input_folder_t2, '.laz'))
files.extend(utl.get_list_of_files(input_folder_t3, '.laz'))

# for each file in files
print(len(files))

8


In [82]:
for f in files[0:1]:
    pcdNodes=[]
    # import pcd
    print(f'processing {f}')
    # laz  = laspy.read(f) #enable this
    print(f'loaded {len(laz.xyz)} points')

    #seperate classes
    for c in class_dict['classes']:
        idx=np.where((laz['classes']==c['id']))
        objects_ids=laz['objects'][idx]        
        class_points=laz.xyz[idx]
        
        #separate objects
        for u in np.unique(objects_ids):
            idx=np.where(objects_ids==u)
            object_pcd=o3d.geometry.PointCloud()
            object_pcd.points=o3d.utility.Vector3dVector(class_points[idx])
            pcdNode=PointCloudNode(resource=object_pcd,
                                    name=ut.get_filename(f)+'_'+c['name']+'_'+str(u),
                                    class_name=c['name'],
                                    class_id=c['id'],
                                    object_id=u,
                                    color=ut.random_color(),
                                    derived_from=ut.get_filename(f))
            pcdNode.get_oriented_bounding_box()
            pcdNodes.append(pcdNode)
    # del laz
    # del class_points
    print(f'created {len(pcdNodes)} from {ut.get_filename(f)}')


processing c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/input/05_MedOffice_01_F2.laz
loaded 44936482 points
created 231 from 05_MedOffice_01_F2


In [85]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in pcdNodes if n.resource is not None])
o3d.visualization.draw_geometries([joined_pcd])

### Import graph (temp)

In [81]:
# files=utl.get_list_of_files(input_folder_t1_temp_graph, '.ttl')

# for f in files[0:1]:
#     meshNodes=[]
#     graph=Graph().parse(f)
#     nodes=tl.graph_to_nodes(graph)
#     for c in class_dict['classes']:
#         meshNodes.append([n for n in nodes if n.class_id==c['id']])
#         print(f'{len(meshNodes[-1])} {c["name"]} detected!')


## OUTPUTS

### export graphs

In [83]:
tl.nodes_to_graph(pcdNodes,
                graphPath=str(output_folder/f'{ut.get_filename(f)}_graph.ttl'),
                save=True)

<Graph identifier=N2f126350df9347f89c42a09fcf3123e9 (<class 'rdflib.graph.Graph'>)>

In [29]:
import torch
input_folder=path/'data'/'t1'/'train' # this should become t1_results
output_folder=path/'data'/'t7'/'inputs'
os.makedirs(output_folder, exist_ok=True)

files=utl.get_list_of_files(input_folder, '.pth')
pcds=[]
for f in files:
    print(f)
    #load pth file
    data=torch.load(f)
    print(data.keys())
    #create a point cloud from the coord and color
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(data['coord'])
    pcd.colors=o3d.utility.Vector3dVector(data['color'])
    #get semantic_gt
    labels=data['semantic_gt']
    
    # get filename
    name=ut.get_filename(f)
    
    #create las file
    header = laspy.LasHeader(point_format=3, version="1.2")
    header.add_extra_dim(laspy.ExtraBytesParams(name="classes", type=np.int32))

    las = laspy.LasData(header)
    las.header.offset = [0, 0, 0]
    las.header.scale = [0.01, 0.01, 0.01]
    las.header.min = [0, 0, 0]
    las.header.max = [1, 1, 1]
    las.x = data['coord'][:,0]
    las.y = data['coord'][:,1]
    las.z = data['coord'][:,2]
    las.red=data['color'][:,0]
    las.green=data['color'][:,1]
    las.blue=data['color'][:,2]
    las.classes=labels
    # gmu.las_add_extra_dimensions(las,(labels),['classes'],['uint8'])
    las.write(f'{output_folder}/{name}.laz')

    #visualize the point cloud
    # pcds.append(PointCloudNode(resource=pcd,color=ut.random_color(),name=ut.get_filename(f)))

c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/train/05_MedOffice_01_F2_small_0_1_0.pth
dict_keys(['coord', 'color', 'normal', 'scene_id', 'semantic_gt'])
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/train/05_MedOffice_01_F2_small_1_1_0.pth
dict_keys(['coord', 'color', 'normal', 'scene_id', 'semantic_gt'])
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/train/19_MedOffice_07_F4_small_0_0_0.pth
dict_keys(['coord', 'color', 'normal', 'scene_id', 'semantic_gt'])
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/train/19_MedOffice_07_F4_small_0_1_0.pth
dict_keys(['coord', 'color', 'normal', 'scene_id', 'semantic_gt'])
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/train/19_MedOffice_07_F4_small_1_0_0.pth
dict_keys(['coord', 'color', 'normal', 'scene_id', 'semantic_gt'])
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVP

In [14]:
np.unique(np.asarray(pcd.colors)[:,0])

array([0.])

In [20]:
print([n.resource for n in pcdNodes])

[PointCloud with 20000000 points., PointCloud with 20000000 points., PointCloud with 4936482 points.]


In [25]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.random_color()) for n in pcdNodes if n.resource is not None])

o3d.visualization.draw_geometries([joined_pcd])
# o3d.visualization.draw_geometries([pcdNodes[0].resource])

